<a href="https://colab.research.google.com/github/MooNight009/ColabProjects/blob/master/DONE_FINE_Bert_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install bert
!pip install nltk
!pip install bert-tensorflow

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_hub as hub

import bert
from bert.tokenization import FullTokenizer

import nltk
import numpy as np
import pandas as pd
import math

nltk.download('punkt')

print("TF version:", tf.__version__)
print("Eager:", "True" if tf.executing_eagerly else "False")
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "not available")

TensorFlow 2.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
TF version: 2.0.0
Eager: True
GPU is not available


In [0]:
max_seq_len= 15

input_word_ids= tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_word_ids")
input_mask= tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_mask")
segment_ids= tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="segment_ids")

bert_layer= hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable= True)

pooled_output, sequence_output= bert_layer([input_word_ids, input_mask, segment_ids])

model= tf.keras.Model(inputs= [input_word_ids, input_mask, segment_ids], outputs= [pooled_output, sequence_output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 15)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 15)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 15)]         0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
def get_masks(tokens, max_seq_len):
  if len(tokens)> max_seq_len:
  
    raise IndexError("Token length is more than max")
  return [1]* len(tokens)+ [0]* (max_seq_len- len(tokens))


def get_segments(tokens, max_seq_len):
  if len(tokens)> max_seq_len:
    raise IndexError("Token length is more than max")
  
  segments= []
  current_segment_id= 0

  for token in tokens:
    segments.append(current_segment_id)

    if token== "[SEP]":
      current_segment_id+= 1
  
  return segments+ [0]* (max_seq_len- len(tokens))


def get_ids(tokens, tokenizer, max_seq_len):
  token_ids= tokenizer.convert_tokens_to_ids(tokens)

  return token_ids+ [0]* (max_seq_len- len(tokens))


# nltk tokenizer has been used to allow for multiple sentences
def tokenize(text):
  text= text.lower()
  sTokens= nltk.sent_tokenize(text)
  wTokens= [nltk.word_tokenize(sen) for sen in sTokens]
  tokens= []
  tokens.append("[CLS]")
  for sen in wTokens:
    for word in sen:
      tokens.append(word)
    tokens.append("[SEP]")

  return tokens

In [0]:
def getPrediction(model, tokens, max_seq_length= 128):
  vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
  do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
  tokenizer = FullTokenizer(vocab_file, do_lower_case)


  input_ids = get_ids(tokens, tokenizer, max_seq_length)
  input_masks = get_masks(tokens, max_seq_length)
  input_segments = get_segments(tokens, max_seq_length)

  # The following was used to bypass shape and len errors
  input_ids= np.array(input_ids)
  input_masks= np.array(input_masks)
  input_segments= np.array(input_segments)

  input_ids= input_ids.reshape(-1, 1)
  input_masks= input_masks.reshape(-1, 1)
  input_segments= input_segments.reshape(-1, 1)

  input_ids= np.transpose(input_ids)
  input_masks= np.transpose(input_masks)
  input_segments= np.transpose(input_segments)


  inp= input_ids, input_masks, input_segments
 

  
  
  pool_embs, all_embs = model.predict(inp)

  return pool_embs, all_embs

In [0]:
text= "This is a nice sentence! which follows this"
# getPrediction(model, tokenize(text), max_seq_length= max_seq_len)